In [61]:
import folium
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from lxml import etree
import requests
from IPython.display import display, HTML
import re


In [62]:
projects= r'P3_GrantExport.csv'
state_data = pd.read_csv(projects, sep=';')
state_data.set_index(state_data.columns[0], inplace=True)
state_data = state_data[['University', 'Approved Amount']]
state_data.dropna(subset=['University'], inplace=True)
state_data = state_data[state_data['University'] != 'Nicht zuteilbar - NA'] #Nicht zuteilbar = NA University
state_data = state_data[state_data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO'] #NPO = is not university

In [117]:

def getData(project_id):
    url = 'http://p3.snf.ch'
    url_project = url + '/project-' + str(project_id)
    # We parse the response to analyze it 
    soup_project = BeautifulSoup(requests.request("GET", url_project).text, 'html.parser')
    id_institute = soup_project.find(id="ctl00_MainContent_urlInstitite").get('href')
    if id_institute is None:
        return None
    
    url_institute = url + id_institute
    
    toSoup = requests.request("GET", url_institute)
    soup_institute = BeautifulSoup(toSoup.text, 'html5lib')
    table = soup_institute.findAll('div', {'class': 'institute'})[0].findAll('td')[1].text
    p = re.compile("CH-\S{4}")
    match = p.search(table)
    
    if match is None:
        return None
    #print(project_id, match.group(0))
    postal_code = match.group(0)
    return postal_code

In [118]:
def search_missing_cantons(UniCanton):
    for i in range(len(UniCanton)):
        if UniCanton['Canton'].iloc[i] == 'NaN':
            query = getData(temp.index.values[i])
            if query is not None:
            canton = searchCantonGeoNames(query)
            UniCanton['Canton'].iloc[i] = canton
    return UniCanton


In [119]:
def searchCantonGeoNames(query1, query2=None): #returns in which canton the university is in
    url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=jnls'
    payload = {'q': query1 }
    req = requests.get(url, params=payload)
    req_json = json.loads(req.text)
    if req_json['totalResultsCount'] > 0:
        return req_json['geonames'][0]['adminCode1']
    elif req_json['totalResultsCount'] == 0 and query2 is not None:
        payload = {'q': query2 }
        req = requests.get(url, params=payload)
        req_json = json.loads(req.text)
        if req_json['totalResultsCount'] > 0:
            return req_json['geonames'][0]['adminCode1']
        else:
            return "NaN"
def search_cantons(data):
    UniCanton = data['University'].drop_duplicates().to_frame()
    UniCanton['Canton'] = ''
    for i in range(len(UniCanton)):
        query = UniCanton['University'].iloc[i].split(" - ")
        if len(query) > 1:
            canton = searchCantonGeoNames(query[0], query[1])
        else:
            canton = searchCantonGeoNames(query[0])
        UniCanton['Canton'].iloc[i] = canton
    #data = data[data['Canton'] != 'NaN'] 
    return UniCanton

temp = search_cantons(state_data)
display(temp)
temp = search_missing_cantons(temp)
display(temp)

,University,Canton
"﻿""Project Number""",,
4,Université de Genève - GE,GE
6,Universität Basel - BS,BS
8,Université de Fribourg - FR,FR
10,Universität Zürich - ZH,ZH
11,Université de Lausanne - LA,VD
30,Universität Bern - BE,BE
39,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",ZH
48,Université de Neuchâtel - NE,NE
60,ETH Zürich - ETHZ,ZH


71 CH-1202
107 CH-6003
115 None
174 CH-7000
260 CH-8090
882 CH-8032
988 CH-1211
1232 CH-8002
2291 CH-9000
2782 CH-7260
3147 CH-3400
3315 CH-2002
5701 CH-6000
5731 CH-7270
5803 CH-7270
5953 CH-5070
6012 CH-4058
6197 CH-8820
6865 CH-6600
7121 CH-1066
10319 CH-3052
10645 CH-8600
10947 CH-6904
11008 CH-7000
15319 None
26632 CH-1951
32399 CH-1920
35819 CH-3005
38024 CH-1950
41977 CH-6048
43157 CH-1950
43389 None
59733 CH-9401
63764 CH-2301
63879 CH-1211
64366 CH-2300
64381 CH-3001
101947 None
108054 CH-1700
108429 CH-6410
109730 CH-8280
112421 CH-8280
118236 CH-5001
119358 CH-1015
120089 CH-3900
124485 CH-9400
127003 CH-6300
127329 CH-6605
127834 CH-6924
132207 CH-8200
133539 CH-3011
134468 CH-4125
139469 CH-6904
143157 CH-3900


,University,Canton
"﻿""Project Number""",,
4,Université de Genève - GE,GE
6,Universität Basel - BS,BS
8,Université de Fribourg - FR,FR
10,Universität Zürich - ZH,ZH
11,Université de Lausanne - LA,VD
30,Universität Bern - BE,BE
39,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",ZH
48,Université de Neuchâtel - NE,NE
60,ETH Zürich - ETHZ,ZH


In [ ]:
temp.set_index(temp.University, inplace=True, drop=True)
temp = temp.to_dict()['Canton']
state_data['Canton'] = state_data['University'].map(temp)
state_data.reset_index(drop=True, inplace=True)
grants = state_data.drop('University', 1)
grants['Approved Amount'] = grants['Approved Amount'].convert_objects(convert_numeric=True)
grants_sum = grants.groupby(grants.Canton).sum()
grant_canton = grants_sum['Approved Amount'].to_dict()


In [172]:
cantonMap = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR', 'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})
cantonMap['Grants'] = 0
cantonMap['Grants'] = cantonMap['Canton'].map(grant_per_canton)

ch_geo = r'ch-cantons.topojson.json'

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=cantonMap, 
                columns=['Canton', 'Grants'], 
                geo_path=ch_geo,
                topojson='objects.cantons', 
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')
map.save('ch-cantons.html')

/Users/dmartr/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
